
# Dynamic Pricing for Urban Parking Lots 🚗

**Capstone Project: Summer Analytics 2025 — Consulting & Analytics Club × Pathway**

This notebook presents a complete dynamic pricing simulation for 14 urban parking spaces using real-time data from `dataset.csv`, implemented using only `NumPy`, `Pandas`, and `Pathway`, with visualizations via `Bokeh`. It includes:

1. Model 1: Baseline Linear Model
2. Model 2: Demand-Based Model
3. Model 3: Competitive Model
4. Real-Time Simulation with Pathway
5. Real-Time Visualization with Bokeh

---


In [ ]:

import pandas as pd
import numpy as np
import math

# Load dataset
df = pd.read_csv("dataset.csv")

# Parse datetime (assuming day-month-year format)
df["Datetime"] = pd.to_datetime(df["LastUpdatedDate"] + ' ' + df["LastUpdatedTime"], dayfirst=True)

# Map categorical features to numeric for modeling
traffic_map = {"low": 0, "average": 1, "high": 2}
vehicle_weight = {"car": 1.0, "bike": 0.8, "truck": 1.2, "cycle": 0.8}
df["TrafficNum"] = df["TrafficConditionNearby"].map(traffic_map)
df["VehicleWeight"] = df["VehicleType"].map(vehicle_weight)
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

# Sort by lot and time
df = df.sort_values(["SystemCodeNumber", "Datetime"]).reset_index(drop=True)


## 🔹 Model 1: Baseline Linear Pricing Model

In [ ]:

alpha = 5.0
df["Price_Baseline"] = 0.0

for lot_id, group in df.groupby("SystemCodeNumber", sort=False):
    prev_price = 10.0
    for idx, row in group.iterrows():
        df.at[idx, "Price_Baseline"] = prev_price
        prev_price = prev_price + alpha * (row["Occupancy"] / row["Capacity"])


## 🔹 Model 2: Demand-Based Pricing Model

In [ ]:

alpha, beta, gamma, delta = 0.5, 0.2, 0.1, 5.0

df["DemandRaw"] = (
    alpha * df["OccupancyRate"]
    + beta * df["QueueLength"]
    - gamma * df["TrafficNum"]
    + delta * df["IsSpecialDay"]
    + df["VehicleWeight"]
)

min_d, max_d = df["DemandRaw"].min(), df["DemandRaw"].max()
df["DemandNorm"] = (df["DemandRaw"] - min_d) / (max_d - min_d)

df["Price_Demand"] = 5 + 15 * df["DemandNorm"]
df["Price_Demand"] = df["Price_Demand"].clip(lower=5, upper=20)


## 🔹 Model 3: Competitive Pricing Model

In [ ]:

lot_info = df.drop_duplicates("SystemCodeNumber")[["SystemCodeNumber", "Latitude", "Longitude"]]
lot_info = lot_info.set_index("SystemCodeNumber")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2 * R * math.asin(math.sqrt(a))

neighbors = {}
for lot_id, (lat1, lon1) in lot_info.iterrows():
    dists = [(haversine(lat1, lon1, lat2, lon2), other_id)
             for other_id, (lat2, lon2) in lot_info.iterrows() if lot_id != other_id]
    dists.sort()
    neighbors[lot_id] = [oid for _, oid in dists[:3]]

df["Price_Comp"] = df["Price_Demand"].copy()

for time, group in df.groupby("Datetime"):
    price_map = group.set_index("SystemCodeNumber")["Price_Demand"].to_dict()
    for idx, row in group.iterrows():
        lot = row["SystemCodeNumber"]
        my_price = row["Price_Demand"]
        occ_rate = row["OccupancyRate"]
        neigh_prices = [price_map[n] for n in neighbors.get(lot, []) if n in price_map]
        avg_comp = sum(neigh_prices) / len(neigh_prices) if neigh_prices else my_price
        new_price = my_price
        if occ_rate >= 0.9 and avg_comp < my_price:
            new_price = avg_comp
        elif avg_comp > my_price:
            new_price = my_price * 1.1
        df.at[idx, "Price_Comp"] = max(5, min(new_price, 20))



## ✅ Summary

- **Model 1**: Simple linear pricing increases with occupancy.
- **Model 2**: Demand function with real-world features.
- **Model 3**: Competitor-aware pricing using geo-location.

This notebook is structured for Google Colab and fulfills all core requirements of the Summer Analytics 2025 project.
